In [ ]:
import duckdb
from pathlib import Path
from settings import settings

In [ ]:
conn = duckdb.connect(Path(settings.get("db_path", "data/alpha_vantage.db")))

In [ ]:
result = conn.execute("SELECT * FROM MACRO").df()
print(result)